# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f5c6cd62cd0>
├── 'a' --> tensor([[-2.5674, -0.7385,  0.9280],
│                   [ 0.7477,  0.9393,  0.4323]])
└── 'x' --> <FastTreeValue 0x7f5c6cd62ca0>
    └── 'c' --> tensor([[-1.0317,  1.6372,  2.3600, -0.6968],
                        [-0.4756,  0.5890, -0.8248,  0.2028],
                        [-0.7974,  0.8075, -0.0083, -0.5645]])

In [4]:
t.a

tensor([[-2.5674, -0.7385,  0.9280],
        [ 0.7477,  0.9393,  0.4323]])

In [5]:
%timeit t.a

61.1 ns ± 0.0747 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f5c6cd62cd0>
├── 'a' --> tensor([[ 2.1547,  2.3485,  1.9156],
│                   [ 0.3708,  0.0410, -1.2556]])
└── 'x' --> <FastTreeValue 0x7f5c6cd62ca0>
    └── 'c' --> tensor([[-1.0317,  1.6372,  2.3600, -0.6968],
                        [-0.4756,  0.5890, -0.8248,  0.2028],
                        [-0.7974,  0.8075, -0.0083, -0.5645]])

In [7]:
%timeit t.a = new_value

61.3 ns ± 0.0402 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-2.5674, -0.7385,  0.9280],
               [ 0.7477,  0.9393,  0.4323]]),
    x: Batch(
           c: tensor([[-1.0317,  1.6372,  2.3600, -0.6968],
                      [-0.4756,  0.5890, -0.8248,  0.2028],
                      [-0.7974,  0.8075, -0.0083, -0.5645]]),
       ),
)

In [10]:
b.a

tensor([[-2.5674, -0.7385,  0.9280],
        [ 0.7477,  0.9393,  0.4323]])

In [11]:
%timeit b.a

56.7 ns ± 0.0482 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.2198, -1.1417, -1.4312],
               [ 0.7967,  0.2565,  1.1930]]),
    x: Batch(
           c: tensor([[-1.0317,  1.6372,  2.3600, -0.6968],
                      [-0.4756,  0.5890, -0.8248,  0.2028],
                      [-0.7974,  0.8075, -0.0083, -0.5645]]),
       ),
)

In [13]:
%timeit b.a = new_value

481 ns ± 0.0296 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

828 ns ± 0.249 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.4 µs ± 22.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

138 µs ± 1.53 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

140 µs ± 403 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f5bb5067820>
├── 'a' --> tensor([[[-2.5674, -0.7385,  0.9280],
│                    [ 0.7477,  0.9393,  0.4323]],
│           
│                   [[-2.5674, -0.7385,  0.9280],
│                    [ 0.7477,  0.9393,  0.4323]],
│           
│                   [[-2.5674, -0.7385,  0.9280],
│                    [ 0.7477,  0.9393,  0.4323]],
│           
│                   [[-2.5674, -0.7385,  0.9280],
│                    [ 0.7477,  0.9393,  0.4323]],
│           
│                   [[-2.5674, -0.7385,  0.9280],
│                    [ 0.7477,  0.9393,  0.4323]],
│           
│                   [[-2.5674, -0.7385,  0.9280],
│                    [ 0.7477,  0.9393,  0.4323]],
│           
│                   [[-2.5674, -0.7385,  0.9280],
│                    [ 0.7477,  0.9393,  0.4323]],
│           
│                   [[-2.5674, -0.7385,  0.9280],
│                    [ 0.7477,  0.9393,  0.4323]]])
└── 'x' --> <FastTreeValue 0x7f5bb5067af0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32.3 µs ± 158 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f5c6cdbae80>
├── 'a' --> tensor([[-2.5674, -0.7385,  0.9280],
│                   [ 0.7477,  0.9393,  0.4323],
│                   [-2.5674, -0.7385,  0.9280],
│                   [ 0.7477,  0.9393,  0.4323],
│                   [-2.5674, -0.7385,  0.9280],
│                   [ 0.7477,  0.9393,  0.4323],
│                   [-2.5674, -0.7385,  0.9280],
│                   [ 0.7477,  0.9393,  0.4323],
│                   [-2.5674, -0.7385,  0.9280],
│                   [ 0.7477,  0.9393,  0.4323],
│                   [-2.5674, -0.7385,  0.9280],
│                   [ 0.7477,  0.9393,  0.4323],
│                   [-2.5674, -0.7385,  0.9280],
│                   [ 0.7477,  0.9393,  0.4323],
│                   [-2.5674, -0.7385,  0.9280],
│                   [ 0.7477,  0.9393,  0.4323]])
└── 'x' --> <FastTreeValue 0x7f5c6cdbadc0>
    └── 'c' --> tensor([[-1.0317,  1.6372,  2.3600, -0.6968],
                        [-0.4756,  0.5890, -0.8248,  0.2028],
                 

In [23]:
%timeit t_cat(trees)

30.7 µs ± 126 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

60.4 µs ± 351 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.0317,  1.6372,  2.3600, -0.6968],
                       [-0.4756,  0.5890, -0.8248,  0.2028],
                       [-0.7974,  0.8075, -0.0083, -0.5645]],
              
                      [[-1.0317,  1.6372,  2.3600, -0.6968],
                       [-0.4756,  0.5890, -0.8248,  0.2028],
                       [-0.7974,  0.8075, -0.0083, -0.5645]],
              
                      [[-1.0317,  1.6372,  2.3600, -0.6968],
                       [-0.4756,  0.5890, -0.8248,  0.2028],
                       [-0.7974,  0.8075, -0.0083, -0.5645]],
              
                      [[-1.0317,  1.6372,  2.3600, -0.6968],
                       [-0.4756,  0.5890, -0.8248,  0.2028],
                       [-0.7974,  0.8075, -0.0083, -0.5645]],
              
                      [[-1.0317,  1.6372,  2.3600, -0.6968],
                       [-0.4756,  0.5890, -0.8248,  0.2028],
                       [-0.7974,  0.8075, -0.0083, -0.5645]],

In [26]:
%timeit Batch.stack(batches)

77.8 µs ± 173 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.0317,  1.6372,  2.3600, -0.6968],
                      [-0.4756,  0.5890, -0.8248,  0.2028],
                      [-0.7974,  0.8075, -0.0083, -0.5645],
                      [-1.0317,  1.6372,  2.3600, -0.6968],
                      [-0.4756,  0.5890, -0.8248,  0.2028],
                      [-0.7974,  0.8075, -0.0083, -0.5645],
                      [-1.0317,  1.6372,  2.3600, -0.6968],
                      [-0.4756,  0.5890, -0.8248,  0.2028],
                      [-0.7974,  0.8075, -0.0083, -0.5645],
                      [-1.0317,  1.6372,  2.3600, -0.6968],
                      [-0.4756,  0.5890, -0.8248,  0.2028],
                      [-0.7974,  0.8075, -0.0083, -0.5645],
                      [-1.0317,  1.6372,  2.3600, -0.6968],
                      [-0.4756,  0.5890, -0.8248,  0.2028],
                      [-0.7974,  0.8075, -0.0083, -0.5645],
                      [-1.0317,  1.6372,  2.3600, -0.6968],
                   

In [28]:
%timeit Batch.cat(batches)

142 µs ± 625 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

304 µs ± 1.87 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
